In [1]:
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.lang.es.examples import sentences
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import string

In [2]:
dataset_train = pd.read_json('dataset_es_train.json', lines = True)
dataset_train.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,es_0359921,product_es_0656090,reviewer_es_0224702,1,"No entro en descalificar al vendedor, solo pue...",PRODUCTO NO RECIBIDO.,es,wireless
4,es_0068940,product_es_0662544,reviewer_es_0224827,1,Llega tarde y co la talla equivocada,Devuelto,es,shoes


In [3]:
np.random #.seed(56)
index_random = np.random.randint(0,high = dataset_train.shape[0])
descripcion = dataset_train.iloc[index_random].review_body
print(index_random, descripcion)

71947 Lo compre porque mi taladro no tenia tope de profundidad para taladrar(antiguo pero sirve todavía). De los tres que vienen monté el que sirve para broca del 6. El tornillo de presión es tan simple que saltó a las pocas operaciones por lo que fue a la basura. Tengo las otras dos que no voy a usar. NO RECOMENDABLE.


In [4]:
nlp = spacy.load("es_core_news_sm")

In [5]:
doc = nlp (descripcion)

In [6]:
for token in doc:
    print(token.text)

Lo
compre
porque
mi
taladro
no
tenia
tope
de
profundidad
para
taladrar(antiguo
pero
sirve
todavía
)
.
De
los
tres
que
vienen
monté
el
que
sirve
para
broca
del
6
.
El
tornillo
de
presión
es
tan
simple
que
saltó
a
las
pocas
operaciones
por
lo
que
fue
a
la
basura
.
Tengo
las
otras
dos
que
no
voy
a
usar
.
NO
RECOMENDABLE
.


In [7]:
stopwords_spacy = list(STOP_WORDS)
print (stopwords_spacy)
len(stopwords_spacy)

['podrias', 'cuatro', 'hago', 'debajo', 'tal', 'he', 'cerca', 'contra', 'siempre', 'sabemos', 'somos', 'adelante', 'podrian', 'cuantas', 'cada', 'qeu', 'aquella', 'lugar', 'peor', 'antes', 'dijo', 'intenta', 'consigue', 'buena', 'ciertas', 'ciertos', 'eramos', 'vamos', 'ella', 'eran', 'unos', 'alli', 'añadió', 'actualmente', 'esa', 'eso', 'menos', 'explicó', 'tras', 'sois', 'aqui', 'suya', 'consigo', 'hecho', 'sido', 'conseguimos', 'ver', 'claro', 'sólo', 'ese', 'más', 'mas', 'donde', 'ante', 'suyo', 'según', 'ambos', 'ningunas', 'aquél', 'gran', 'aseguró', 'tuyo', 'conmigo', 'podría', 'mediante', 'usais', 'aquéllas', 'respecto', 'emplean', 'dos', 'quizas', 'voy', 'segunda', 'sera', 'modo', 'como', 'tampoco', 'del', 'bajo', 'saber', 'aquellas', 'pronto', 'mismas', 'ningún', 'primera', 'llegó', 'ustedes', 'cuánta', 'eras', 'hizo', 'ultimo', 'momento', 'sé', 'sabeis', 'si', 'todas', 'se', 'empleo', 'él', 'mismo', 'esto', 'mencionó', 'dentro', 'vosotras', 'realizar', 'través', 'haciendo',

551

In [8]:
for token in doc:
    if token.is_stop == False:
        print(token)

compre
taladro
tenia
tope
profundidad
taladrar(antiguo
sirve
)
.
vienen
monté
sirve
broca
6
.
tornillo
presión
simple
saltó
a
operaciones
a
basura
.
a
.
RECOMENDABLE
.


In [9]:
for token in doc:
    print (token.text, token.lemma_)

Lo él
compre comprar
porque porque
mi mi
taladro taladro
no no
tenia teniar
tope tope
de de
profundidad profundidad
para para
taladrar(antiguo taladrar(antiguo
pero pero
sirve servir
todavía todavía
) )
. .
De de
los el
tres tres
que que
vienen venir
monté montar
el el
que que
sirve servir
para para
broca broca
del del
6 6
. .
El el
tornillo tornillo
de de
presión presión
es ser
tan tanto
simple simple
que que
saltó saltar
a a
las el
pocas poco
operaciones operación
por por
lo él
que que
fue ser
a a
la el
basura basura
. .
Tengo tener
las el
otras otro
dos dos
que que
no no
voy ir
a a
usar usar
. .
NO no
RECOMENDABLE recomendable
. .


In [10]:
for token in doc:
    print (token.text, token.pos_)

Lo PRON
compre VERB
porque SCONJ
mi DET
taladro NOUN
no ADV
tenia VERB
tope NOUN
de ADP
profundidad NOUN
para ADP
taladrar(antiguo NOUN
pero CCONJ
sirve VERB
todavía ADV
) PUNCT
. PUNCT
De ADP
los DET
tres NUM
que PRON
vienen VERB
monté VERB
el DET
que PRON
sirve VERB
para ADP
broca NOUN
del ADP
6 NUM
. PUNCT
El DET
tornillo NOUN
de ADP
presión NOUN
es AUX
tan ADV
simple ADJ
que SCONJ
saltó VERB
a ADP
las DET
pocas DET
operaciones NOUN
por ADP
lo PRON
que PRON
fue AUX
a ADP
la DET
basura NOUN
. PUNCT
Tengo VERB
las DET
otras DET
dos NUM
que PRON
no ADV
voy AUX
a ADP
usar VERB
. PUNCT
NO ADV
RECOMENDABLE ADJ
. PUNCT


In [11]:
puntua = string.punctuation + '¿!¡? + " "'
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿!¡? + " "'

In [12]:
puntua = string.punctuation + '¿!¡? + " "'
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿!¡? + " "'

In [13]:
def text_data_cleaning(sentence):
    doc=nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp= token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
        
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
            
    return " ".join(clean_tokens)

In [14]:
text_data_cleaning(descripcion)

'comprar taladro teniar tope profundidad taladrar(antiguo servir venir montar servir broca 6 tornillo presión simple saltar a operación a basura a recomendable'

In [15]:
dataset_train.review_body.apply(text_data_cleaning)

0         kar pantalla 8 mes y recibir respuesta fabricante
1         Horrible comprar inglés informático hora capaz...
2         obligar a comprar unidad y llegar y forma recl...
3         entrar descalificar vendedor mes espera .... s...
4                              llegar y co talla equivocado
                                ...                        
199995    Mando funcionar perfectamente y cumplir funció...
199996    Compré batería reticencia resultar fácil insta...
199997                            calidad satisfecho compra
199998                                 Perfecto cumple hijo
199999                    Súper brocha caer pelito chula xd
Name: review_body, Length: 200000, dtype: object

In [16]:
dataset_train.head(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore


In [17]:
dataset_train['review_body']= dataset_train.review_body.apply(text_data_cleaning)

In [19]:
dataset_train

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0038754,product_es_0113523,reviewer_es_0580071,1,llegar articulo mandar correo normal seguimien...,no me llego,es,wireless
1,es_0748979,product_es_0017036,reviewer_es_0819733,1,mensajería horrible compro,amazon sigue sin cumplir en las entregas,es,home
2,es_0411746,product_es_0138642,reviewer_es_0508607,1,decepcionado vendedor pedido llegar a y cumpli...,ESTAFA EN EL ENVÍO,es,toy
3,es_0786686,product_es_0170887,reviewer_es_0491157,1,valoración producto AMAZON ofreceis producto a...,Estafa de Amazon,es,home
4,es_0429700,product_es_0710642,reviewer_es_0008745,1,interés libro y probar versión kindle abrir po...,No conseguí pasar de la portada en Kindle,es,digital_ebook_purchase
...,...,...,...,...,...,...,...,...
4995,es_0685611,product_es_0919143,reviewer_es_0461769,5,costar perfecto cumplir función sobra peque ca...,Perfectos para lo que sirven,es,wireless
4996,es_0638242,product_es_0636134,reviewer_es_0214562,5,material flexible y cómodo acorde a descripción,"Buen vendedor, rápido y fiable.",es,wireless
4997,es_0760108,product_es_0078583,reviewer_es_0043609,5,grabado,medallas con bonito grabado,es,home
4998,es_0070806,product_es_0359350,reviewer_es_0258717,5,comprar bici finalmente quedar xiaomi scooter ...,timbre,es,sports


In [26]:
dataset_train.to_csv('dataset_train.csv', index = False)

In [18]:
dataset_train

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,kar pantalla 8 mes y recibir respuesta fabricante,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,Horrible comprar inglés informático hora capaz...,Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,obligar a comprar unidad y llegar y forma recl...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,es_0359921,product_es_0656090,reviewer_es_0224702,1,entrar descalificar vendedor mes espera .... s...,PRODUCTO NO RECIBIDO.,es,wireless
4,es_0068940,product_es_0662544,reviewer_es_0224827,1,llegar y co talla equivocado,Devuelto,es,shoes
...,...,...,...,...,...,...,...,...
199995,es_0715276,product_es_0317036,reviewer_es_0643604,5,Mando funcionar perfectamente y cumplir funció...,Tal y como se describe,es,electronics
199996,es_0085190,product_es_0622919,reviewer_es_0466173,5,Compré batería reticencia resultar fácil insta...,Funciona perfectamente,es,electronics
199997,es_0484496,product_es_0358101,reviewer_es_0330744,5,calidad satisfecho compra,Buena calidad.,es,apparel
199998,es_0930141,product_es_0788855,reviewer_es_0694290,5,Perfecto cumple hijo,Recomendado,es,toy
